In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  StandardScaler,MinMaxScaler
from sklearn.neighbors import  KNeighborsClassifier,KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV,cross_val_score
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

In [124]:
bankdata=pd.read_excel(r"C:\Users\Prarthana Saikia\Desktop\Praxis\Machine Learning\ML class\codes for exam\datasets\bank marketing\bank.xls")

In [125]:
bankdata.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [107]:
from scipy.stats import chi2_contingency
cont1 = pd.crosstab(bankdata['job'],bankdata['deposit'])
print(scipy.stats.chi2_contingency(cont1))

NameError: name 'scipy' is not defined

In [68]:
#checking null values

bankdata.isnull().sum()
bankdata.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

In [80]:
#set the target variable

target=bankdata.iloc[:,-1]
y = (target == 'yes').astype(np.int64)

In [5]:
y.dtypes

dtype('int64')

In [126]:
#drop the target variable

bankdata = bankdata.drop(['deposit'], axis = 1)

In [127]:
#taking out numerical data, some columns even with num data had less values, so considered them as ordinal data
X_num = bankdata.iloc[:,[0,5,11,13,14]]

In [128]:
X_num.head()

,age,balance,duration,pdays,previous
0,59,2343,1042,-1,0
1,56,45,1467,-1,0
2,41,1270,1389,-1,0
3,55,2476,579,-1,0
4,54,184,673,-1,0


In [129]:
#dropping the values which were taken earlier as num data
bankdata = bankdata.drop(['balance', 'age', 'duration','pdays','previous' ], axis = 1)

In [130]:
bankdata.head()

,job,marital,education,default,housing,loan,contact,day,month,campaign,poutcome
0,admin.,married,secondary,no,yes,no,unknown,5,may,1,unknown
1,admin.,married,secondary,no,no,no,unknown,5,may,1,unknown
2,technician,married,secondary,no,yes,no,unknown,5,may,1,unknown
3,services,married,secondary,no,yes,no,unknown,5,may,1,unknown
4,admin.,married,tertiary,no,no,no,unknown,5,may,2,unknown


In [131]:
X_cat=bankdata.iloc[:,:]
X_cat.head()

,job,marital,education,default,housing,loan,contact,day,month,campaign,poutcome
0,admin.,married,secondary,no,yes,no,unknown,5,may,1,unknown
1,admin.,married,secondary,no,no,no,unknown,5,may,1,unknown
2,technician,married,secondary,no,yes,no,unknown,5,may,1,unknown
3,services,married,secondary,no,yes,no,unknown,5,may,1,unknown
4,admin.,married,tertiary,no,no,no,unknown,5,may,2,unknown


In [132]:
#converting to dummies
X_cat.drop(['day','month'],axis=1,inplace=True)
#from sklearn.preprocessing import LabelEncoder
#encode = LabelEncoder()
#X_cat = X_cat.apply(encode.fit_transform)

In [133]:
X_cat.head()

,job,marital,education,default,housing,loan,contact,campaign,poutcome
0,admin.,married,secondary,no,yes,no,unknown,1,unknown
1,admin.,married,secondary,no,no,no,unknown,1,unknown
2,technician,married,secondary,no,yes,no,unknown,1,unknown
3,services,married,secondary,no,yes,no,unknown,1,unknown
4,admin.,married,tertiary,no,no,no,unknown,2,unknown


In [134]:
from sklearn.preprocessing import LabelEncoder
encode=LabelEncoder()
X_cat = X_cat.apply(encode.fit_transform)
#X_out1= X_out.apply(encode.fit_transform)
X_cat.head()

,job,marital,education,default,housing,loan,contact,campaign,poutcome
0,0,1,1,0,1,0,2,0,3
1,0,1,1,0,0,0,2,0,3
2,9,1,1,0,1,0,2,0,3
3,7,1,1,0,1,0,2,0,3
4,0,1,2,0,0,0,2,1,3


In [135]:
#chi squared for feature selection

#performing chi2 test for feature selection 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
sel = SelectKBest(chi2,3)#this number 12 is a rsult after some hit and trials
a=sel.fit(X_cat, y)
score = (sel.scores_)
score

array([ 103.96072735,   16.64600454,   44.90924103,   18.19363512,
        244.47443048,  118.63540388,  953.66745096,  880.20789124,
         67.84877579])

In [136]:
X_cat.shape

(11162, 9)

In [137]:
X_cat = sel.transform(X_cat)

In [138]:
X_cat.shape

(11162, 3)

In [139]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators = 300)

In [140]:
X = np.concatenate((X_cat,X_num), axis = 1)

In [141]:
#splitting of data into training set and test set
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(  X, y, test_size=2000, random_state=42)

In [142]:
#fitting the model on training set
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [143]:
#for evaluation
from sklearn.metrics import confusion_matrix as cm
matrix =  cm(y_test,clf1.predict(X_test) )

#checking accuracy score
accuracy = (matrix[0,0] + matrix[1,1])/2000
print(accuracy)

0.8005
